In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("LR").getOrCreate()

In [ ]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/cruise.csv >> cruise.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8734  100  8734    0     0  61943      0 --:--:-- --:--:-- --:--:-- 61943


In [ ]:
df = spark.read.csv("/content/cruise.csv", header= True, inferSchema= True)

In [ ]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
df.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")

In [ ]:
indexed = indexer.fit(df).transform(df)

In [ ]:
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

In [ ]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler =VectorAssembler(
    inputCols=[
            'Age',
            'Tonnage',
            'passengers',
            'length',
            'cabins',
            'passenger_density',
            'cruise_cat'],
    outputCol="features"
)

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [ ]:
final_data = output.select("features", "crew")

In [ ]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [ ]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed = 24)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol="crew")

In [ ]:
model= lr.fit(train_data)

In [ ]:
res = model.evaluate(test_data)

In [ ]:
res.r2

0.9721372591819774

In [ ]:
res.rootMeanSquaredError

0.4783645381106928

In [ ]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/e-com_customer.csv >> ecom.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 86871  100 86871    0     0   415k      0 --:--:-- --:--:-- --:--:--  415k


In [ ]:
data = spark.read.csv("/content/ecom.csv", inferSchema=True, header= True)

In [ ]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(
    inputCols= ['Avg Session Length',
                'Time on App',
                'Time on Website',
                'Length of Membership',
              ],
    outputCol="features"
)

In [ ]:
output=assembler.transform(data)

In [ ]:
final_data = output.select("features", "Yearly Amount Spent")

In [ ]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed = 71)

In [ ]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [ ]:
lrmodel = lr.fit(train_data)

In [ ]:
res = lrmodel.evaluate(test_data)

In [ ]:
res.r2

0.9855533130052111

In [ ]:
res.r2

0.982486819515339

In [ ]:
!curl https://raw.githubusercontent.com/TakMashhido/PGD-BigData-Tutorial/main/Dataset/drug.csv >> drug.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5827  100  5827    0     0  32553      0 --:--:-- --:--:-- --:--:-- 32553


In [ ]:
data = spark.read.csv('/content/drug.csv', inferSchema= True, header = True)

In [ ]:
data.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- Cholesterol: string (nullable = true)
 |-- Na_to_K: double (nullable = true)
 |-- Drug: string (nullable = true)



In [ ]:
data.show()

+---+---+------+-----------+-------+-----+
|Age|Sex|    BP|Cholesterol|Na_to_K| Drug|
+---+---+------+-----------+-------+-----+
| 23|  F|  HIGH|       HIGH| 25.355|drugY|
| 47|  M|   LOW|       HIGH| 13.093|drugC|
| 47|  M|   LOW|       HIGH| 10.114|drugC|
| 28|  F|NORMAL|       HIGH|  7.798|drugX|
| 61|  F|   LOW|       HIGH| 18.043|drugY|
| 22|  F|NORMAL|       HIGH|  8.607|drugX|
| 49|  F|NORMAL|       HIGH| 16.275|drugY|
| 41|  M|   LOW|       HIGH| 11.037|drugC|
| 60|  M|NORMAL|       HIGH| 15.171|drugY|
| 43|  M|   LOW|     NORMAL| 19.368|drugY|
| 47|  F|   LOW|       HIGH| 11.767|drugC|
| 34|  F|  HIGH|     NORMAL| 19.199|drugY|
| 43|  M|   LOW|       HIGH| 15.376|drugY|
| 74|  F|   LOW|       HIGH| 20.942|drugY|
| 50|  F|NORMAL|       HIGH| 12.703|drugX|
| 16|  F|  HIGH|     NORMAL| 15.516|drugY|
| 69|  M|   LOW|     NORMAL| 11.455|drugX|
| 43|  M|  HIGH|       HIGH| 13.972|drugA|
| 23|  M|   LOW|       HIGH|  7.298|drugC|
| 32|  F|  HIGH|     NORMAL| 25.974|drugY|
+---+---+--

In [ ]:
data.columns

['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug']

In [ ]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in list(set(data.columns)-set(['Age', 'Na_to_K' ]))]

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=indexer)

In [ ]:
df_d = pipeline.fit(data).transform(data)

In [ ]:
df_d.show()

+---+---+------+-----------+-------+-----+-----------------+--------+----------+---------+
|Age|Sex|    BP|Cholesterol|Na_to_K| Drug|Cholesterol_index|BP_index|Drug_index|Sex_index|
+---+---+------+-----------+-------+-----+-----------------+--------+----------+---------+
| 23|  F|  HIGH|       HIGH| 25.355|drugY|              0.0|     0.0|       0.0|      1.0|
| 47|  M|   LOW|       HIGH| 13.093|drugC|              0.0|     1.0|       4.0|      0.0|
| 47|  M|   LOW|       HIGH| 10.114|drugC|              0.0|     1.0|       4.0|      0.0|
| 28|  F|NORMAL|       HIGH|  7.798|drugX|              0.0|     2.0|       1.0|      1.0|
| 61|  F|   LOW|       HIGH| 18.043|drugY|              0.0|     1.0|       0.0|      1.0|
| 22|  F|NORMAL|       HIGH|  8.607|drugX|              0.0|     2.0|       1.0|      1.0|
| 49|  F|NORMAL|       HIGH| 16.275|drugY|              0.0|     2.0|       0.0|      1.0|
| 41|  M|   LOW|       HIGH| 11.037|drugC|              0.0|     1.0|       4.0|      0.0|

In [ ]:
df_d.columns

['Age',
 'Sex',
 'BP',
 'Cholesterol',
 'Na_to_K',
 'Drug',
 'Cholesterol_index',
 'BP_index',
 'Drug_index',
 'Sex_index']

In [ ]:
assembler = VectorAssembler(
    inputCols = ['Age',
                'Na_to_K',
                'Cholesterol_index',
                'BP_index',
                'Sex_index'],
    outputCol = "features"
)

In [ ]:
output = assembler.transform(df_d)

In [ ]:
output.show()

+---+---+------+-----------+-------+-----+-----------------+--------+----------+---------+--------------------+
|Age|Sex|    BP|Cholesterol|Na_to_K| Drug|Cholesterol_index|BP_index|Drug_index|Sex_index|            features|
+---+---+------+-----------+-------+-----+-----------------+--------+----------+---------+--------------------+
| 23|  F|  HIGH|       HIGH| 25.355|drugY|              0.0|     0.0|       0.0|      1.0|[23.0,25.355,0.0,...|
| 47|  M|   LOW|       HIGH| 13.093|drugC|              0.0|     1.0|       4.0|      0.0|[47.0,13.093,0.0,...|
| 47|  M|   LOW|       HIGH| 10.114|drugC|              0.0|     1.0|       4.0|      0.0|[47.0,10.114,0.0,...|
| 28|  F|NORMAL|       HIGH|  7.798|drugX|              0.0|     2.0|       1.0|      1.0|[28.0,7.798,0.0,2...|
| 61|  F|   LOW|       HIGH| 18.043|drugY|              0.0|     1.0|       0.0|      1.0|[61.0,18.043,0.0,...|
| 22|  F|NORMAL|       HIGH|  8.607|drugX|              0.0|     2.0|       1.0|      1.0|[22.0,8.607,0.

In [ ]:
final_data = output.select("features", "Drug_index")

In [ ]:
final_data.show()

+--------------------+----------+
|            features|Drug_index|
+--------------------+----------+
|[23.0,25.355,0.0,...|       0.0|
|[47.0,13.093,0.0,...|       4.0|
|[47.0,10.114,0.0,...|       4.0|
|[28.0,7.798,0.0,2...|       1.0|
|[61.0,18.043,0.0,...|       0.0|
|[22.0,8.607,0.0,2...|       1.0|
|[49.0,16.275,0.0,...|       0.0|
|[41.0,11.037,0.0,...|       4.0|
|[60.0,15.171,0.0,...|       0.0|
|[43.0,19.368,1.0,...|       0.0|
|[47.0,11.767,0.0,...|       4.0|
|[34.0,19.199,1.0,...|       0.0|
|[43.0,15.376,0.0,...|       0.0|
|[74.0,20.942,0.0,...|       0.0|
|[50.0,12.703,0.0,...|       1.0|
|[16.0,15.516,1.0,...|       0.0|
|[69.0,11.455,1.0,...|       1.0|
|(5,[0,1],[43.0,13...|       2.0|
|[23.0,7.298,0.0,1...|       4.0|
|[32.0,25.974,1.0,...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

In [ ]:
train_data, test_data = final_data.randomSplit([0.8,0.2], seed = 27)

In [ ]:
dt = DecisionTreeClassifier(labelCol="Drug_index", featuresCol="features")
rf = RandomForestClassifier(labelCol="Drug_index", featuresCol="features")

In [ ]:
dt_model = dt.fit(train_data)

In [ ]:
rf_model =rf.fit(train_data)

In [ ]:
dt_res =dt_model.transform(test_data)
rf_res= rf_model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Drug_index", predictionCol="prediction", metricName="accuracy")

In [ ]:
dt_acc = acc_evaluator.evaluate(dt_res)
rf_acc = acc_evaluator.evaluate(rf_res)

In [ ]:
dt_acc*100

97.82608695652173

In [ ]:
rf_acc*100

100.0